**NAMED ENTITY RECOGNITION**

URN: 6693898

We are using Frames dataset which has 1369 conversations of human. Frames dialogue was compiled in a Wizard of Oz style manner. A chat interface was used to communicate between two persons. One takes the role of the user and the other, wizard who is a conversational agent. 

We can find the dataset in detail here: https://www.microsoft.com/en-us/research/project/frames-dataset/

All the conversations happening are stored in the 'turns' column of the dataset.

In [14]:
import pandas as pd

dataset = pd.read_json('frames.json')
len(dataset)

1369

In [15]:
dataset["turns"][0][0]

{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'active_frame': 1,
  'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
     'budget': [{'val': '1700.0', 'negated': False}],
     'dst_city': [{

Named Entity Recongnition helps in extracting the entities from a text i.e divides the text into pre defined categories like cities, individuals,organization,etc. 

SpaCy is a open and free source of Python library which is used for text classification which is one of the most basic application of NLP. It has a good NER accuracy of 85.85%. 

In order to work with SpaCy, we need to transform the data into SpaCy format:

[(input text, entites:[(start_index, end_index, entity_name), (start_index, end_index, entity_name), ...]), ....]



**EXTRACTING TEXT AND ENTITIES PRESENT IN THAT TEXT**

In [20]:
key_value = dict()

data = []


for conversation in dataset['turns']:
    for index, turn in enumerate(conversation):
        if turn['author'] == 'user':
            
        
            dataitem = set()
            utterance = turn["text"]
            dataitem.add(utterance)
            #print(utterance)

            for act in turn['labels']['acts']:

                entities = []


                for arg in act["args"]:

                    # No value for the key, useless
                    if "val" not in arg:
                        continue

                    #Not considering key_value pairs with annotations 
                    if "val" in arg and type(arg["val"]) == list:
                        continue


                    #INCLUDE KEYS HERE WHICH YOU DON"T WANT IN THE TRAINING DATASET
                    #
                    if arg["key"] in ["intent", "flex", "id", "max_duration"]:
                        continue


                    #print(arg)

                    start_index = utterance.find(str(arg["val"]))
                    end_index = start_index + len(str(arg["val"])) - 1
                    #print(start_index)
                    #print(end_index)

                    entities.append((start_index, end_index, arg["key"],)) 

                #print(entities)
                if len(entities) > 0:
                    data.append((utterance, {"entities": entities}))
    

data   

[("I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  {'entities': [(27, 34, 'dst_city'),
    (41, 47, 'or_city'),
    (52, 76, 'str_date'),
    (82, 82, 'n_adults'),
    (117, 120, 'budget')]}),
 ('Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.',
  {'entities': [(24, 32, 'dst_city'),
    (70, 70, 'n_adults'),
    (114, 117, 'budget')]}),
 ('I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?',
  {'entities': [(56, 63, 'or_city')]}),
 ('Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.',
  {'entities': [(59, 68, 'dst_city'),
    (44, 54, 'or_city'),
    (75, 78, 'budget')]}),
 ('What about a trip from Gotham City to Neverland for the same budget?',
  {'entities': [(38, 46, 'dst_city'), (23, 33, 'or_city')]}),
 ('Would any packages to Mos Eisley be available if

In [21]:
len(data)

5849

**Installing dependecies**

In [22]:
! pip install jsonlines
! pip install -U spacy -q 
! python -m spacy info

2022-05-19 12:48:01.671794: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:48:01.671846: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

============================== Info about spaCy ==============================

spaCy version    3.3.0                         
Location         /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/spacy
Platform         Linux-4.14.276-211.499.amzn2.x86_64-x86_64-with-glibc2.31
Python version   3.9.10                        
Pipelines        en_core_web_sm (3.3.0)        



In [23]:
! pip install spacy[transformers]

In [24]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import re
import jsonlines
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object


2022-05-19 12:49:02.143418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:49:02.143453: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.





We will start with Training the Model. We have taken first 24 datas for training. 

In [25]:
TRAIN_DATA =[
    ("I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  {'entities': [(27, 34, 'dst_city'),
    (41, 47, 'or_city'),
    (52, 76, 'str_date'),
    (82, 82, 'n_adults'),
    (117, 120, 'budget')]}),
 ('Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.',
  {'entities': [(24, 32, 'dst_city'),
    (70, 70, 'n_adults'),
    (114, 117, 'budget')]}),
 ('I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?',
  {'entities': [(56, 63, 'or_city')]}),
 ('Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.',
  {'entities': [(59, 68, 'dst_city'),
    (44, 54, 'or_city'),
    (75, 78, 'budget')]}),
 ('What about a trip from Gotham City to Neverland for the same budget?',
  {'entities': [(38, 46, 'dst_city'), (23, 33, 'or_city')]}),
 ('Would any packages to Mos Eisley be available if I increase my budget to $2500?',
  {'entities': [(22, 31, 'dst_city'), (73, 77, 'budget')]}),
 ("You know what, I'd like to try and visit Neverland",
  {'entities': [(41, 49, 'dst_city')]}),
 ('Do you have any trips from Gotham City to Kobe for my original budget of $2100?',
  {'entities': [(42, 45, 'dst_city')]}),
 ("No, that's too far for me. I need a flight that leaves from Birmingham.",
  {'entities': [(60, 69, 'or_city')]}),
 ('Hello there i am looking to go on a vacation with my family to Gotham City, can you help me?',
  {'entities': [(63, 73, 'dst_city')]}),
 ('Not sure when we want to leave, but we are 12 kids and 5 adults',
  {'entities': [(-1, 0, 'str_date'),
    (55, 55, 'n_adults'),
    (43, 44, 'n_children')]}),
 ('yes i do, it is around $2200', {'entities': [(23, 27, 'budget')]}),
 ('We are from Neverland', {'entities': [(12, 20, 'or_city')]}),
 ('we can depart from Toronto', {'entities': [(19, 25, 'or_city')]}),
 ('hmm what options would i have out of Toronto?',
  {'entities': [(37, 43, 'or_city')]}),
 ("Hi I'd like to go to Caprica from Busan, between Sunday August 21, 2016 and Wednesday August 31, 2016",
  {'entities': [(21, 27, 'dst_city'),
    (34, 38, 'or_city'),
    (49, 70, 'str_date'),
    (76, 100, 'end_date')]}),
 ("Actually it's unlimited for this trip", {'entities': [(-1, 0, 'budget')]}),
 ('2 adults', {'entities': [(0, 0, 'n_adults')]}),
 ('Do you have anything for San Antonio as a destination?',
  {'entities': [(25, 35, 'dst_city')]}),
 ('Hello, I am looking to book a trip for 2 adults and 6 children for $21,300 or less. We are departing from Kochi for Denver.',
  {'entities': [(52, 52, 'n_children'),
    (39, 39, 'n_adults'),
    (67, 73, 'budget'),
    (106, 110, 'or_city'),
    (116, 121, 'dst_city')]}),
 ('I do not have any dates in mind. I would like to spend as much time in Denver as my budget will allow.',
  {'entities': [(-1, 0, 'str_date'), (-1, 0, 'end_date')]}),
 ('Hey, i Want to go to St. Louis on the 17th of August',
  {'entities': [(21, 29, 'dst_city'), (38, 51, 'str_date')]}),
 ('I need to back by the 31st', {'entities': [(22, 25, 'end_date')]}),
 ('I’m from Calgary', {'entities': [(9, 15, 'or_city')]})
]

We would be using Spacy's Spancategorizer along with the SpaCy NER to measure the confidence score of the NER model as it can potentially overlap and predicts scored and labels for the asked spans where as NER model takes single token based tags. 

**Creating Training Data for Spacy's NER and Spancategorizer**


In [26]:
for text, annot in tqdm(TRAIN_DATA): 
    doc = re.sub(' +', ' ', text)
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    print(text)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    try:
      ents = []
      for start, end, label in annot["entities"]: # add character indexes
          span = doc.char_span(start, end, label=label, alignment_mode="strict")
          if span is not None:
              ents.append(span)            
      doc.ents = ents # label the text with the ents
    except Exception as e:
      doc.ents = []
    # spacy.displacy.render(doc, style="ent", jupyter=True)
    db.add(doc)

db.to_disk("./spandata.spacy") # save the docbin object

#after getting ner data converting into doc.spans["sc"]
docbin = DocBin().from_disk("./spandata.spacy")
docs = list(docbin.get_docs(nlp.vocab))
for doc in docs:
    doc.spans["sc"] = list(doc.ents)
DocBin(docs=docs).to_disk("./spansdata.spacy")

nlp.add_pipe('spancat')

100%|██████████| 24/24 [00:00<00:00, 707.88it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
Skipping entity
Skipping entity
Skipping entity
Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.
Skipping entity
I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?
Skipping entity
Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.
Skipping entity
What about a trip from Gotham City to Neverland for the same budget?
Would any packages to Mos Eisley be available if I increase my budget to $2500?
Skipping entity
You know what, I'd like to try and visit Neverland
Skipping entity
Do you have any trips from Gotham City to Kobe for my original budget of $2100?
Skipping entity
No, that's too far for me. I need a flight that leaves from Birmingham.
Hello there i 

100%|██████████| 24/24 [00:00<00:00, 6863.72it/s]


**Loading the model with default settings and creating the config file with NER and spancat**

In [27]:
! python -m spacy init config config.cfg --lang en --pipeline  ner,spancat --optimize efficiency --force 

2022-05-19 12:49:52.336965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:49:52.337004: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner, spancat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


**Initializing the training**

In [28]:
! python -m spacy train config.cfg --output ./NER --paths.train ./spansdata.spacy --paths.dev ./spansdata.spacy 

2022-05-19 12:50:01.237574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:50:01.237608: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Saving to output directory: NER
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-19 12:50:03,883] [INFO] Set up nlp object from config
[2022-05-19 12:50:03,893] [INFO] Pipeline: ['tok2vec', 'ner', 'spancat']
[2022-05-19 12:50:03,897] [INFO] Created vocabulary
[2022-05-19 12:50:03,898] [INFO] Finished initializing nlp object
[2022-05-19 12:50:04,124] [INFO] Initialized pipeline components: ['tok2vec', 'ner', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner', 'spa

Standard metrics for evaluating dependency parsing are UAS(Unlabelled Attachment Score) and LAS(Labelled Attachment Score). The proportion of tokens whose heads have been correctly assignmed is known as UAS, while the proportion of tokens whose heads have been correctly assigned with the right dependence label is known as LAS(subject,object,etc). 

The precision,recall and fscore for NER job are-ents p,ents r and ents f.
Aditionally, these are calculated on each entity basis.That is ,when searching for true positives,false positives and false negatives, spaCy evaluates all entities in our data. 

In [29]:
nlp = spacy.load("./NER/model-best/") 
doc =nlp( 'Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.')

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter



As we can see above, the performance of the model is not so good as it is giving an accuracy of 55% and only extracting some entities.

We will try changing the hyperparameter for optimization i.e changing the learning rate. By default the learning rate was set to 0.001 or 1e-3 . We will change it to 0.0001 or 1e-4 to see if there's any improvement. We are doing that from the config.cfg file

In [30]:
! python -m spacy init config learning_rate_1e-4_config.cfg --lang en --pipeline  ner,spancat --optimize efficiency --force  

2022-05-19 12:54:21.312485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:54:21.312522: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner, spancat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
learning_rate_1e-4_config.cfg
You can now add your data and train your pipeline:
python -m spacy train learning_rate_1e-4_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [31]:
! python -m spacy train config.cfg --output ./NER --paths.train ./spansdata.spacy --paths.dev ./spansdata.spacy 

2022-05-19 12:54:56.410169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:54:56.410204: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Saving to output directory: NER
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-19 12:54:58,969] [INFO] Set up nlp object from config
[2022-05-19 12:54:58,979] [INFO] Pipeline: ['tok2vec', 'ner', 'spancat']
[2022-05-19 12:54:58,983] [INFO] Created vocabulary
[2022-05-19 12:54:58,983] [INFO] Finished initializing nlp object
[2022-05-19 12:54:59,192] [INFO] Initialized pipeline components: ['tok2vec', 'ner', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner', 'spa

In [32]:
nlp = spacy.load("./NER/model-best/") 
doc =nlp("I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.")

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

We can observed that after changing the learning rate from 0.001 to 0.0001 , the model improved the score from 55% to 63% which means that the model will try to learn slowly. Still, the entities labelled are not correct. Therefore, the model needs to be trained further.


**We will try gathering more data for training and validation and check if its makes any difference(fulltraining approach)**

In [33]:
TRAIN_DATA= data

In [34]:
for text, annot in tqdm(TRAIN_DATA): 
    doc = re.sub(' +', ' ', text)
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    print(text)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    try:
      ents = []
      for start, end, label in annot["entities"]: # add character indexes
          span = doc.char_span(start, end, label=label, alignment_mode="strict")
          if span is not None:
              ents.append(span)            
      doc.ents = ents # label the text with the ents
    except Exception as e:
      doc.ents = []
    # spacy.displacy.render(doc, style="ent", jupyter=True)
    db.add(doc)

db.to_disk("./spandata.spacy") # save the docbin object

#after getting ner data converting into doc.spans["sc"]
docbin = DocBin().from_disk("./spandata.spacy")
docs = list(docbin.get_docs(nlp.vocab))
for doc in docs:
    doc.spans["sc"] = list(doc.ents)
DocBin(docs=docs).to_disk("./spansdata.spacy")


  3%|▎         | 159/5849 [00:00<00:03, 1579.83it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.
Skipping entity
Skipping entity
Skipping entity
Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.
Skipping entity
I have no flexibility for dates... but I can leave from Atlantis rather than Caprica. How about that?
Skipping entity
Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.
Skipping entity
What about a trip from Gotham City to Neverland for the same budget?
Would any packages to Mos Eisley be available if I increase my budget to $2500?
Skipping entity
You know what, I'd like to try and visit Neverland
Skipping entity
Do you have any trips from Gotham City to Kobe for my original budget of $2100?
Skipping entity
No, that's too far for me. I need a flight that leaves from Birmingham.
Hello there i 

  8%|▊         | 455/5849 [00:00<00:04, 1313.23it/s]

Skipping entity
I have business in Calgary around those dates, what say I leave from Calgary, could I get a trip somewhere?
Skipping entity
Yes, thank you. I prefer business class for this trip.
Skipping entity
Im leaving from Queenstown
I really want to go to Sao Paulo is there anything there?
August 17, 2016 to Wednesday, August 31, 2016 are the dates i could be gone
Skipping entity
can we go ahed and book that?
Skipping entity
HI there! my family and i are looking to go to Burlington
Skipping entity
Skipping entity
1 child and 6 adults
Skipping entity
Campinnas
Skipping entity
ya sure that sounds great! and how much would that be for business?
Hi im looking to book  trip. anything leaving from Kakariko Village?
Skipping entity
termina perhaps?
Skipping entity
Anything going to Caprica
Skipping entity
Skipping entity
Good afternoon, my man! I'd like to plan a honeymoon in Naples for 5 days leaving from Madrid on August 17. What is available?
Skipping entity
Hi i want to plan my honey

 13%|█▎        | 767/5849 [00:00<00:03, 1456.61it/s]

...

 91%|█████████▏| 5341/5849 [00:02<00:00, 2547.59it/s]

Skipping entity
Skipping entity
7! Big family, I know HAHA. We will be leaving from Porto and I should probably mention we can only afford trips under 17200. Do you have anything between September 7th and 25th?
Skipping entity
not bad. My wife wants me to explore my options so do you have anything to Birmingham?
Okay, okay. Well do you have anything to San Juan?
Hey there, I'm excited to book a trip somewhere for me and my colleagues.
Skipping entity
There will be 27 of us.
Skipping entity
We're in Beijing we're thinking of going to Santo Domingo.
Skipping entity
Skipping entity
No, because all 27 of us live in Beijing.
Skipping entity
Heyyy! I would like to book an awesome trip for me and ma gurls from Fort Lauderdale to Minneapolis.
Skipping entity
3 adults
Skipping entity
Skipping entity
Skipping entity
September 7th to 19th! and we've got 2800 between the 3 of us.
Skipping entity
Perf! Do you have anything to Phoenix? We heard its a fall break hot spot!
Skipping entity
Is that for 

100%|██████████| 5849/5849 [00:02<00:00, 2215.59it/s]

Skipping entity
In a last attempt to find something.. can you see if there are better trips to Marseille?
Skipping entity
Skipping entity
Skipping entity
I would like to take my wife and 4 kids on a trip from Toronto to Porto
How bout to  La Paz?
Skipping entity
What about Cleveland?
Skipping entity
Skipping entity
Skipping entity
Skipping entity
I need to find MEWTWO!!!!! I need to get to Paris from Kingston on or after the 6th. I have a friend tagging along to help me and we have 6000
Skipping entity
Not bad, a Cathedral might be a good Pokéstop. Do you have anything to Phoenix?
Skipping entity
Skipping entity
Skipping entity
Skipping entity
I need to leave Beijing, I’d like to go to Munich with 10 adults and 3300 between us
Skipping entity
Skipping entity
How bout Kabul to Munich?
Skipping entity
Skipping entity
Any flights to New York for 2300 and 4 adults?
Skipping entity
Kabul
Skipping entity
Ok then leave from Beijing
Skipping entity
Skipping entity
I want to take my five friend


100%|██████████| 5849/5849 [00:00<00:00, 10055.00it/s]


In [35]:
! python -m spacy init config config.cfg --lang en --pipeline  ner,spancat --optimize efficiency --force  

2022-05-19 12:59:15.309575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 12:59:15.309610: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner, spancat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [37]:
! python -m spacy train config.cfg --output ./NER --paths.train ./spansdata.spacy --paths.dev ./spansdata.spacy 

2022-05-19 13:05:44.353858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 13:05:44.353896: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Saving to output directory: NER
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-19 13:05:46,930] [INFO] Set up nlp object from config
[2022-05-19 13:05:46,940] [INFO] Pipeline: ['tok2vec', 'ner', 'spancat']
[2022-05-19 13:05:46,944] [INFO] Created vocabulary
[2022-05-19 13:05:46,945] [INFO] Finished initializing nlp object
[2022-05-19 13:06:05,219] [INFO] Initialized pipeline components: ['tok2vec', 'ner', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner', 'spa

On training the whole data, we can see that the accuracy went down which shows that the provided data is not good enough. That is there's a lot of room for improvement.

We are now changing the optimizer for Adam.v1 to RAdam.v1 to see there is any improvements. Rectified Adam(RAdam) can obtain greater accuracy or at least least comparable accuracy to Adam and in fewer epochs than ordinary Adam.

In [48]:
! python -m spacy init config RAdam_v1_config.cfg --lang en --pipeline  ner,spancat --optimize efficiency --force  

2022-05-19 13:30:16.538333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 13:30:16.538370: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner, spancat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
RAdam_v1_config.cfg
You can now add your data and train your pipeline:
python -m spacy train RAdam_v1_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [50]:
! python -m spacy train RAdam_v1_config.cfg --output ./NER --paths.train ./spansdata.spacy --paths.dev ./spansdata.spacy 

2022-05-19 13:46:04.331552: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 13:46:04.331589: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ℹ Saving to output directory: NER
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-19 13:46:06,929] [INFO] Set up nlp object from config
[2022-05-19 13:46:06,940] [INFO] Pipeline: ['tok2vec', 'ner', 'spancat']
[2022-05-19 13:46:06,944] [INFO] Created vocabulary
[2022-05-19 13:46:06,945] [INFO] Finished initializing nlp object
[2022-05-19 13:46:25,287] [INFO] Initialized pipeline components: ['tok2vec', 'ner', 'spancat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner', 'spa


RAdam is slower than Adam techniques in the first few cycles of rectifying items,but it is faster in the later stages of convergence.

**Code which takes text as the input and labels entities along with the text as an output**

In [60]:
a = input(" ")


  Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.


In [61]:
nlp = spacy.load("./NER/model-best/") 
doc = nlp(a)
span_group = doc.spans["sc"] # default key, can be changed
scores = span_group.attrs["scores"]

# Note that `scores` is an array with one score for each span in the group
for span, score, ent in zip(span_group, scores, doc.ents):
    print("LABEL :",span.label_,"start :","span :", span, "len :",len(span.text),"score : ",score,ent.start_char,ent.end_char  )

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

From my observation I concluded that by changing the learing rate from 1e-3 to 1e-4 made a small improvement in the accuracy. But, when we trained model with full data, it still has less accuracy which might have happened due to model overfitting. The model must have memorized a lot that is causing high error rates for unseen datas. 

The training of the data has not been nice. The quality of the data should be qood enough for the model to learn better and produce some quality outputs.In order to make the NER, either the data should be hand labelled or its done by using unsupervised learning. 




**REFERENCES**

https://spacy.io/

https://spacy.io/usage/training

https://www.researchgate.net/post/Why_the_accuracy_of_the_model_decreases_with_increase_in_data_set

https://github.com/chawla201/Custom-Named-Entity-Recognition/blob/main/train_model.ipynb

https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718

https://pyimagesearch.com/2019/10/07/is-rectified-adam-actually-better-than-adam/

https://spacy.io/api/spancategorizer
